In [4]:
# 항상 첫 번째 셀
import sys
import os
sys.path.append(os.path.abspath("../src"))

In [ ]:
def crawl_musinsa_goods(category_code="103", max_pages=1000):
    """
    무신사 API를 통해 상품 ID와 썸네일 URL을 수집합니다.
    :param category_code: 수집할 카테고리 코드 (예: 001)
    :param max_pages: 수집할 최대 페이지 수
    :return: 수집된 데이터 리스트
    """
    
    collected_data = []
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Referer": "https://www.musinsa.com/",
        "Accept": "application/json"
    }

    print(f"🚀 크롤링 시작: 카테고리 {category_code}, 최대 {max_pages} 페이지")

    # tqdm을 사용하여 진행률 바 생성
    pbar = tqdm(range(1, max_pages + 1), unit="page")
    
    for page in pbar:
        # 진행 상황 텍스트 업데이트
        pbar.set_description(f"수집 중... Page {page}")

        params = {
            "gf": "M",
            "sortCode": "SALE_ONE_YEAR_COUNT",
            "category": category_code,
            "size": 60,
            "testGroup": "",
            "caller": "CATEGORY",
            "page": page,
            "seen": 0,
            "seenAds": ""
        }
        
        url = "https://api.musinsa.com/api2/dp/v1/plp/goods"

        try:
            response = requests.get(url, headers=headers, params=params, timeout=10)
            
            if response.status_code == 200:
                json_data = response.json()
                goods_list = json_data.get('data', {}).get('list', [])
                
                if not goods_list:
                    pbar.write(f"🛑 [Page {page}] 더 이상 상품이 없습니다. 수집을 종료합니다.")
                    break

                current_items = 0
                for item in goods_list:
                    item_info = {
                        "goodsNo": item.get("goodsNo"),
                        "thumbnail": item.get("thumbnail"),
                        "goodsName": item.get("goodsName"),
                        "brandName": item.get("brandName")
                    }
                    collected_data.append(item_info)
                    current_items += 1
                
                # 우측에 수집된 총 개수 표시 업데이트
                pbar.set_postfix(total_collected=len(collected_data), last_count=current_items)

            else:
                pbar.write(f"⚠️ [Page {page}] 에러 발생: Status {response.status_code}")
                break

        except Exception as e:
            pbar.write(f"❌ [Page {page}] 요청 실패: {e}")
            break

        # 랜덤 딜레이
        time.sleep(random.uniform(0.2, 0.5))

    return collected_data

In [ ]:
from db_client import RDSClient
# from db_client import RDSClient # 이미 import 되어 있다고 가정

# DB 클라이언트 초기화 (필요하다면)
db_client = RDSClient()

def insert_product_data_with_mapping(data_list):
    """
    딕셔너리 리스트 형태의 데이터를 DB에 삽입하기 전에 키를 매핑합니다.
    (goodsNo -> product_id, brandName -> brand, thumbnail -> img_url)
    """
    if not isinstance(data_list, list):
        print("❌ 입력된 데이터가 리스트 타입이 아닙니다.")
        return

    # 1. 삽입 쿼리 정의 (기존과 동일)
    # product_id, brand, img_url 컬럼에 삽입합니다.
    insert_query = """
    INSERT INTO product_shoes_img (product_id, brand, img_url)
    VALUES (:product_id, :brand, :img_url);
    """
    
    total_count = len(data_list)
    success_count = 0
    print(f"🔄 총 {total_count}개의 데이터를 삽입 시도합니다.")

    # 2. 리스트를 순회하며 각 딕셔너리(로우)를 매핑 후 삽입
    for item in data_list:
        
        # 🚨 핵심 수정 부분: 키 매핑 딕셔너리 생성
        try:
            mapped_params = {
                'product_id': item['goodsNo'],
                'brand': item['brandName'],
                'img_url': item['thumbnail']
            }
        except KeyError as e:
            print(f"⚠️ 데이터 딕셔너리에 필요한 키 ({e})가 없습니다. 해당 로우 건너뜀: {item}")
            continue

        # 3. 쿼리 실행
        # 매핑된 딕셔너리(mapped_params)를 파라미터로 사용합니다.
        row_count = db_client.execute(insert_query, mapped_params)
        
        if row_count == 1:
            success_count += 1
        else:
            # 기본 키 중복 등의 이유로 삽입 실패할 수 있습니다.
            print(f"⚠️ product_id {mapped_params['product_id']} 삽입 실패. (영향 받은 행: {row_count})")

    print(f"\n--- 작업 완료 ---")
    print(f"총 {total_count}개 중 {success_count}개의 행을 성공적으로 삽입했습니다.")


In [ ]:
from curl_cffi import requests
import time
import random
import pandas as pd
from tqdm import tqdm  # 진행률 바 라이브러리 추가
from db_client import RDSClient  # 제공해 주신 모듈 import

# --- 실행부 ---
if __name__ == "__main__":
    # 테스트를 위해 페이지 수 설정 (예: 10페이지)
    target_category = "103"
    target_pages = 1000
    result_list = crawl_musinsa_goods(category_code=target_category, max_pages=target_pages) # 응답 리스트: {id, }
  

In [ ]:
result_list

In [ ]:
# ⭐️ 최적화된 방식 (Batch Insert)
import os
from sqlalchemy import create_engine, text
def optimized_insert_product_data(data_list):
    # ... (데이터 매핑은 동일하게 수행하여 mapped_data_list를 만듭니다)
    
    # 1. 쿼리 정의 (기존과 동일)
    insert_query = """
    INSERT INTO product_shoes_img (product_id, brand, img_url)
    VALUES (:product_id, :brand, :img_url);
    """
    
    # 2. 모든 데이터를 매핑하여 하나의 리스트로 준비
    mapped_data_list = []
    for item in data_list:
        mapped_data_list.append({
            'product_id': item['goodsNo'],
            'brand': item['brandName'],
            'img_url': item['thumbnail']
        })
    # 중복 제거를 위한 딕셔너리 생성
    # (나중에 나오는 데이터가 이전 데이터를 덮어씌웁니다)
    unique_data = {}
    
    for item in data_list:
        p_id = item['goodsNo']  # 기준이 되는 ID
        unique_data[p_id] = {
            'product_id': p_id,
            'brand': item['brandName'],
            'img_url': item['thumbnail']
        }
    
    # 2. 중복이 제거된 값들만 리스트로 변환
    deduplicated_list = list(unique_data.values())
    
    # db_client 모듈을 가정하고, 임시로 connection을 직접 사용한다고 가정
    if not db_client.engine: return
    
    print("🔄 배치 삽입 (executemany) 시도...")
    try:
        with db_client.engine.connect() as connection:
            # text()로 쿼리 준비
            stmt = text(insert_query)
            
            # executemany로 한 번에 실행
            result = connection.execute(stmt, deduplicated_list)
            
            # 한 번의 커밋으로 모든 변경사항 반영
            connection.commit() 
            
            print(f"✅ 배치 삽입 성공! 총 {result.rowcount}개 행 삽입.")
    except Exception as e:
        print(f"❌ 배치 삽입 오류 발생: {e}")

# **이 방식은 수 분 내외 (수십 초 ~ 몇 분)로 작업을 완료할 수 있습니다.**

In [ ]:
optimized_insert_product_data(result_list)

In [ ]:
from sqlalchemy import create_engine, text

# 1. 설정
OLD_DB = "musinsa"
NEW_DB = "musinsa_backup_20251213"  # 새로 만들 DB 이름

# DB 연결 (데이터베이스 이름 없이 연결해야 새 DB를 생성할 수 있음)
root_url = f"mysql+pymysql://{"admin"}:{"qkqajrwkrnrnrn9_"}@{"musinsa-data.c07kuo6ug98z.us-east-1.rds.amazonaws.com"}:{3306}"
root_engine = create_engine(root_url)

with root_engine.connect() as conn:
    # 2. 새 스키마(데이터베이스) 생성
    print(f"🔨 새 데이터베이스 '{NEW_DB}' 생성 중...")
    conn.execute(text(f"CREATE DATABASE IF NOT EXISTS {NEW_DB}"))
    
    # 3. 기존 DB의 테이블 목록 가져오기
    print(f"📋 '{OLD_DB}'의 테이블 목록 조회 중...")
    result = conn.execute(text(f"SHOW TABLES FROM {OLD_DB}"))
    tables = [row[0] for row in result]
    
    # 외래 키 제약 조건 임시 해제 (순서 상관없이 복사하기 위해)
    conn.execute(text("SET FOREIGN_KEY_CHECKS=0"))

    # 4. 테이블 하나씩 복제 (구조 + 데이터)
    for table in tqdm(tables, desc="테이블 복제 중"):
        try:
            # (A) 구조 복사 (인덱스 포함)
            # CREATE TABLE new.table LIKE old.table 문법 사용
            conn.execute(text(f"CREATE TABLE IF NOT EXISTS {NEW_DB}.{table} LIKE {OLD_DB}.{table}"))
            
            # (B) 데이터 복사
            # INSERT INTO new.table SELECT * FROM old.table
            conn.execute(text(f"INSERT INTO {NEW_DB}.{table} SELECT * FROM {OLD_DB}.{table}"))
            
        except Exception as e:
            print(f"⚠️ Error copying {table}: {e}")

    # 외래 키 제약 조건 다시 활성화
    conn.execute(text("SET FOREIGN_KEY_CHECKS=1"))
    
    print(f"✅ 전체 스키마 복제 완료! ({OLD_DB} ➡️ {NEW_DB})")

In [ ]:
# [Cell] '올드머니' 페르소나 데이터 삽입
from db_client import RDSClient

db = RDSClient()

# 1. 삽입할 데이터 정의
persona_name = '올드머니'

raw_data = {
    '아우터': [885715, 3481219, 2100931],
    '상의': [3019653, 2139143, 994588],
    '하의': [3154647, 2147933, 1803235],
    '신발': [636821, 3568935, 4770924]
}

if db.engine:
    print(f"--- '{persona_name}' 데이터 준비 중 ---")
    
    # 2. 배치 처리를 위한 파라미터 리스트 변환
    insert_params = []
    
    for category, product_ids in raw_data.items():
        for pid in product_ids:
            insert_params.append({
                'product_id': pid,
                'persona': persona_name,
                'category': category
            })
            
    print(f"준비된 데이터: 총 {len(insert_params)}건")
    
    # 3. DB 삽입 (Upsert 방식)
    # product_id가 PK이므로, 이미 존재하면 persona와 category를 업데이트합니다.
    insert_sql = """
    INSERT INTO persona_items (product_id, persona, category)
    VALUES (:product_id, :persona, :category)
    ON DUPLICATE KEY UPDATE
        persona = VALUES(persona),
        category = VALUES(category)
    """
    
    try:
        # 배치 실행
        result_count = db.execute_batch(insert_sql, insert_params)
        
        if result_count is not None:
            print(f"✅ 삽입 완료! 총 {result_count}행이 처리되었습니다.")
            
            # 4. 결과 확인용 조회
            check_sql = f"""
                SELECT * FROM persona_items 
                WHERE persona = '{persona_name}'
            """
            rows = db.execute(check_sql)
            print(f"\n[DB 저장 결과 확인 - {len(rows)}건]")
            for row in rows:
                print(f"상품ID: {row['product_id']} | 카테고리: {row['category']} | 스타일: {row['persona']}")
                
    except Exception as e:
        print(f"❌ 데이터 삽입 실패: {e}")
        print("💡 팁: product_id가 'products' 테이블에 실제로 존재하는지 확인해보세요. (외래키 제약조건)")

else:
    print("❌ DB 연결 실패")

In [3]:
import numpy as np

# 파일 로드
data = np.load('/Users/siu/Desktop/top_embedded.npz')

# 1. 키 목록 확인 (저장할 때 쓴 변수명들이 나옵니다)
print("저장된 키 목록:", data.files)
# 출력 예상: ['embeddings', 'ids']

# 2. 내용물 꺼내기
loaded_ids = data['ids']
loaded_vecs = data['embeddings']

id_vector_pairs = list(zip(loaded_ids, loaded_vecs))

# 확인
print(f"총 {len(id_vector_pairs)}개의 쌍이 생성되었습니다.")
print(f"첫 번째 데이터 예시: ID={id_vector_pairs[0][0]}, Vector shape={id_vector_pairs[0][1].shape}")

저장된 키 목록: ['embeddings', 'ids']
총 59877개의 쌍이 생성되었습니다.
첫 번째 데이터 예시: ID=70061, Vector shape=(512,)


In [10]:
# [Cell] 스타일별 아이템 이미지 갤러리 생성
from db_client import RDSClient
from IPython.display import display, HTML

# 1. 데이터 정의
persona_name = '올드머니'
raw_data = {
    '아우터': [885715, 3481219, 2100931],
    '상의': [3019653, 2139143, 994588],
    '하의': [3154647, 2147933, 1803235],
    '신발': [636821, 3568935, 4770924]
}

db = RDSClient()

if db.engine:
    # --- 1단계: 모든 ID 수집 및 DB 조회 ---
    all_ids = []
    for ids in raw_data.values():
        all_ids.extend(ids)
    
    # SQL IN 절을 위해 튜플로 변환 (ID가 하나일 경우 오류 방지)
    if len(all_ids) == 1:
        ids_sql_str = f"({all_ids[0]})"
    else:
        ids_sql_str = str(tuple(all_ids))
        
    query = f"""
        SELECT product_id, img_url, product_name 
        FROM products 
        WHERE product_id IN {ids_sql_str}
    """
    
    try:
        rows = db.execute(query)
        
        # --- 2단계: ID -> 정보 매핑 (Lookup Dictionary 생성) ---
        # DB에서 가져온 순서는 보장되지 않으므로, ID로 즉시 찾을 수 있게 딕셔너리로 만듭니다.
        item_map = {row['product_id']: row for row in rows}
        
        # --- 3단계: HTML 생성 ---
        html_out = f"""
        <div style="font-family: sans-serif; padding: 10px;">
            <h2 style="border-bottom: 2px solid #333; padding-bottom: 10px;">
                🎨 Persona: <span style="color: #007bff;">{persona_name}</span>
            </h2>
        """
        
        for category, pids in raw_data.items():
            html_out += f"<h3 style='margin-top: 20px; color: #555;'>🏷️ {category}</h3>"
            html_out += "<div style='display: flex; gap: 15px; flex-wrap: wrap;'>"
            
            for pid in pids:
                item = item_map.get(pid)
                
                if item:
                    img_src = item['img_url']
                    p_name = item.get('product_name', '상품명 없음')
                else:
                    # DB에 해당 ID가 없는 경우
                    img_src = "https://via.placeholder.com/150?text=No+Data"
                    p_name = "DB 미존재"

                # 카드 스타일 HTML
                html_out += f"""
                <div style="
                    border: 1px solid #ddd; 
                    border-radius: 8px; 
                    padding: 10px; 
                    width: 140px; 
                    text-align: center; 
                    box-shadow: 2px 2px 5px rgba(0,0,0,0.1);
                    background: white;">
                    
                    <div style="
                        width: 100%; 
                        height: 160px; 
                        overflow: hidden; 
                        border-radius: 4px; 
                        margin-bottom: 8px;
                        background-color: #f8f9fa;">
                        <img src="{img_src}" style="width: 100%; height: 100%; object-fit: cover;" loading="lazy">
                    </div>
                    
                    <div style="font-size: 12px; font-weight: bold; color: #333;">ID: {pid}</div>
                    <div style="
                        font-size: 11px; 
                        color: #666; 
                        white-space: nowrap; 
                        overflow: hidden; 
                        text-overflow: ellipsis;
                        margin-top: 4px;">
                        {p_name}
                    </div>
                </div>
                """
            html_out += "</div>" # Row 닫기
            
        html_out += "</div>" # Container 닫기
        
        # --- 4단계: 화면 출력 ---
        display(HTML(html_out))
        
    except Exception as e:
        print(f"❌ 데이터 조회 중 오류 발생: {e}")
else:
    print("❌ DB 연결 실패")

✅ DB Engine (Pool) 생성 완료
